In [1]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
from warnings import warn
import shutil
from custom_steps import (
    step_resnet18_tidy,
    step_resnet18_streamline,
    step_resnet18_convert_to_hls,
    step_resnet18_set_fifo_depths,
    step_resnet18_slr_floorplan)

model_file = "models/resnet18.onnx"
board = "KV260_SOM"
final_output_dir = "build-{}/{}".format(board,model_file.split('.')[0].split('/')[1])

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")
    
resnet18_build_steps = [
    step_resnet18_tidy,
    step_resnet18_streamline,
    step_resnet18_convert_to_hls,
    "step_create_dataflow_partition",
    "step_apply_folding_config",
    "step_generate_estimate_reports",
    "step_hls_codegen",
    "step_hls_ipgen",
    step_resnet18_set_fifo_depths,
    step_resnet18_slr_floorplan,
    "step_synthesize_bitfile",
    "step_make_pynq_driver",
    "step_deployment_package",
]

try:
    from finn.transformation.fpgadataflow.infer_doublepacked_dsp import InferDoublePackedConv
    folding_config_file="folding_config/U250_folding_config.json"
    print("DoublePackedConv detected")
except:
    warn(" FINN Experimental not available. Using non-packed folded down convolution. This is 16 times slower per MHz ")
    folding_config_file="folding_config/U250_folding_config_no_doublepack_pe_folded_16.json"


cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    steps               = resnet18_build_steps,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    folding_config_file =  folding_config_file,
    board               = board,
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

<ipython-input-1-dcf5e4ffec12>:43: UserWarning:  FINN Experimental not available. Using non-packed folded down convolution. This is 16 times slower per MHz 
  warn(" FINN Experimental not available. Using non-packed folded down convolution. This is 16 times slower per MHz ")


In [ ]:
build.build_dataflow_cfg(model_file, cfg)

Building dataflow accelerator from models/resnet18.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in build-KV260_SOM/resnet18
Build log is at build-KV260_SOM/resnet18/build_dataflow.log
Running step: step_resnet18_tidy [1/13]
Running step: step_resnet18_streamline [2/13]


Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/shares/bulk/pgeel/FINNv0.8.1_repo/TY_build_KV260/finn/notebooks/MSC_Git_files/Resnet-18/custom_steps.py", line 178, in step_resnet18_streamline
    model = step_resnet18_streamline_nonlinear(model, cfg)
  File "/shares/bulk/pgeel/FINNv0.8.1_repo/TY_build_KV260/finn/notebooks/MSC_Git_files/Resnet-18/custom_steps.py", line 169, in step_resnet18_streamline_nonlinear
    model = model.transform(trn)
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/finn/src/finn/transformation/streamline/reorder.py", line 556, in apply
    init0 = model.get_initializer(prod0.input[1])
IndexError: list index (1) out of range


> /workspace/finn/src/finn/transformation/streamline/reorder.py(556)apply()
    554                 if prod0 is None or prod1 is None or (prod0 == prod1):
    555                     continue
--> 556                 init0 = model.get_initializer(prod0.input[1])
    557                 init1 = model.get_initializer(prod1.input[1])
    558                 # if either initializer is None, skip

